# Displacement field NWES
Following the tutorial of https://docs.oceanparcels.org/en/latest/examples/documentation_unstuck_Agrid.html   
we create a displacement field for the CMEMS data of NWES https://data.marine.copernicus.eu/product/NWSHELF_ANALYSISFORECAST_PHY_004_013/description  
we use the new NWES data (from september 2023 onwards)

In [66]:
# import needed packages


from copy import copy

import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
import numpy.ma as ma
import xarray as xr
from matplotlib.colors import ListedColormap
from matplotlib.lines import Line2D
from netCDF4 import Dataset
from scipy import interpolate

import parcels

In [26]:
# set directories and files
base_directory_in = '/storage/shared/oceanparcels/input_data/CMEMS/NORTHWESTSHELF_ANALYSIS_FORECAST_PHY_004_013/'
filename_in_new = 'CMEMS_v6r1_NWS_PHY_NRT_NL_01hav3D_20230901_20230901_R20230902_HC01.nc'
filename_in_old = 'metoffice_foam1_amm15_NWS_CUR_b20230903_hi20230901.nc' 
# there is a old (till nov 2023) and new (from sep 2023) version
versions = np.array(['old','new'])
models = {}
models['old'] = {}
models['new'] = {}
models['old']['data'] = xr.open_dataset(base_directory_in+filename_in_old)
models['new']['data'] = xr.open_dataset(base_directory_in+filename_in_new)

print(models['new']['data'].attrs['source'])
print(models['old']['data'].attrs['source'])
#

#

NEMO3.6
PS-OS 45, AMM-FOAM 1.5 km (tidal) NEMO v3.6_WAVEWATCH-III v4.18_NEMOVAR v6


In [28]:
# settings models
dlon = 1 / 36
dlat = 1 / 36

print(models['new']['data'])

<xarray.Dataset> Size: 10GB
Dimensions:    (time: 24, longitude: 936, latitude: 551, depth: 50)
Coordinates:
  * time       (time) datetime64[ns] 192B 2023-09-01T00:30:00 ... 2023-09-01T...
  * longitude  (longitude) float32 4kB -16.0 -15.97 -15.94 ... 9.921 9.949 9.977
  * latitude   (latitude) float32 2kB 46.0 46.03 46.06 ... 61.23 61.25 61.28
  * depth      (depth) float32 200B 0.494 1.541 2.646 ... 5.275e+03 5.728e+03
Data variables:
    uo         (time, depth, latitude, longitude) float32 2GB ...
    vo         (time, depth, latitude, longitude) float32 2GB ...
    thetao     (time, depth, latitude, longitude) float32 2GB ...
    so         (time, depth, latitude, longitude) float32 2GB ...
Attributes: (12/13)
    Conventions:     CF-1.8
    title:           Ocean 3D hourly mean fields for the North West Shelf (NW...
    source:          NEMO3.6
    domain_name:     NWS36
    field_type:      mean
    field_date:      20230901
    ...              ...
    forecast_type:   hindcas

In [65]:
# resolution old and new model are not the same
models['old']['dlon'] = models['old']['data']['lon'].values[11] - models['old']['data']['lon'].values[10]
models['old']['dlat'] = models['old']['data']['lat'].values[11] - models['old']['data']['lat'].values[10]



print(f'dlon new = 1/{1/models['new']['dlon']:.0f}')
print(f'dlat new = 1/{1/models['new']['dlat']:.0f}')
print(f'dlon old = 1/{1/models['old']['dlon']:.0f}')
print(f'dlat old = 1/{1/models['old']['dlat']:.0f}')

models['old']['lonmin']=models['old']['data']['lon'].values[0]
models['old']['lonmax']=models['old']['data']['lon'].values[-1]
models['old']['latmin']=models['old']['data']['lat'].values[0]
models['old']['latmax']=models['old']['data']['lat'].values[-1]



gridlimits = np.array(['lonmin','lonmax','latmin','latmax'])
for version in versions:
    for g in gridlimits:
        print(f'{version} {g} = {models[version][g]:.2f}')








dlon new = 1/36
dlat new = 1/36
dlon old = 1/33
dlat old = 1/74
old lonmin = -16.00
old lonmax = 13.00
old latmin = 46.00
old latmax = 62.74
new lonmin = -16.00
new lonmax = 9.98
new latmin = 46.00
new latmax = 61.28


In [67]:
# new
models['new']['dlon'] = models['new']['data']['longitude'].values[11] - models['new']['data']['longitude'].values[10]
models['new']['dlat'] = models['new']['data']['latitude'].values[11] - models['new']['data']['latitude'].values[10]
models['new']['lonmin']=models['new']['data']['longitude'].values[0]
models['new']['lonmax']=models['new']['data']['longitude'].values[-1]
models['new']['latmin']=models['new']['data']['latitude'].values[0]
models['new']['latmax']=models['new']['data']['latitude'].values[-1]

x = models['new']['data']["longitude"][:-1] + np.diff(models['new']['data']["longitude"]["longitude"]) / 2
y = models['new']['data']["latitude"][:-1] + np.diff(models['new']['data']["latitude"]) / 2
lon_centers, lat_centers = np.meshgrid(x, y)
models['new']['lon_centers'] = lon_centers
models['new']['lat_centers'] = lat_centers
color_land = copy(plt.get_cmap("Reds"))(0)
color_ocean = copy(plt.get_cmap("Reds"))(128)



In [68]:
def make_landmask(fielddata):
    """Returns landmask where land = 1 and ocean = 0
    fielddata is a netcdf file.
    """
    datafile = Dataset(fielddata)

    landmask = datafile.variables["uo"][0, 0]
    landmask = np.ma.masked_invalid(landmask)
    landmask = landmask.mask.astype("int")

    return landmask